# Hybrid Gradient Boosting Trees  Example via Unified Classification

A data set that identifies whether or not a pentient has diabetes is used to demonstrate the use of hybrid graident boosting classifier in SAP HANA.



# Pima Indians Diabetes Dataset

Original data comes from National Institute of Diabetes and Digestive and Kidney Diseases. The collected dataset is aiming at, based on certain diagnostic measurements, diagnostically predicting whether or not a patient has diabetes. In particular, patients contained in the dataset are females of Pima Indian heritage, all above the age of 20. Dataset is form Kaggle, for tutorials use only.

The dataset contains the following diagnositic <b>attributes</b>:<br>
$\rhd$ "PREGNANCIES" - Number of times pregnant,<br>
$\rhd$ "GLUCOSE" - Plasma glucose concentration a 2 hours in an oral glucose tolerance test,<br>
$\rhd$ "BLOODPRESSURE" -  Diastolic blood pressure (mm Hg),<br>
$\rhd$ "SKINTHICKNESS" -  Triceps skin fold thickness (mm),<br>
$\rhd$ "INSULIN" - 2-Hour serum insulin (mu U/ml),<br>
$\rhd$ "BMI" - Body mass index $(\text{weight in kg})/(\text{height in m})^2$,<br>
$\rhd$ "PEDIGREE" - Diabetes pedigree function,<br>
$\rhd$ "AGE" -  Age (years),<br>
$\rhd$ "CLASS" - Class variable (0 or 1) 268 of 768 are 1(diabetes), the others are 0(non-diabetes).



In [ ]:
import hana_ml
from hana_ml import dataframe
from hana_ml.algorithms.pal import metrics
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification

# Load Data

The data is loaded into 3 tables - full set, training-validation set, and test set as follows:

<li> PIMA_INDIANS_DIABETES_TBL</li>
<li> PIMA_INDIANS_DIABETES_TRAIN_VALID_TBL</li>
<li> PIMA_INDIANS_DIABETES_TEST_TBL</li>

To do that, a connection is created and passed to the loader.

There is a config file, <b>config/e2edata.ini</b> that controls the connection parameters and whether or not to reload the data from scratch. In case the data is already loaded, there would be no need to load the data. A sample section is below. If the config parameter, reload_data is true then the tables for test, training and validation are (re-)created and data inserted into them.

#########################<br>
[hana]<br>
url=host.sjc.sap.corp<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
#########################<br>

In [ ]:
from hana_ml.algorithms.pal.utility import DataSets, Settings
import plotting_utils
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
full_set, diabetes_train, diabetes_test, _ = DataSets.load_diabetes_data(connection_context)

In [ ]:
from hana_ml.algorithms.pal.utility import Settings

Settings.set_log_level()

# Simple Exploration

Let us look at the number of rows in each dataset:

In [ ]:
print('Number of rows in training set: {}'.format(diabetes_train.count()))
print('Number of rows in testing set: {}'.format(diabetes_test.count()))

Let us look at columns of the dataset:

In [ ]:
print(diabetes_train.columns)

Let us also look some (in this example, the top 6) rows of the dataset:

In [ ]:
diabetes_train.head(6).collect()

We can also check the data type of all columns:

In [ ]:
diabetes_train.dtypes()

We have a 'CLASS' column in the dataset, let us check how many classes are contained in this dataset:

In [ ]:
diabetes_train.distinct('CLASS').collect()

Two classes are available, assuring that this is a binary classification problem.

#  Model Creation & Model Selection
The lines below show the ease with which classification can be done.

Set up the label column, use default feature set and create the model:

In [ ]:
cv_values = {}
cv_values['learning_rate'] = [0.1, 0.4, 0.7, 1.0]
cv_values['n_estimators'] = [4, 6, 8, 10]
cv_values['split_threshold'] = [0.1, 0.4, 0.7, 1.0]
hgc = UnifiedClassification(func='HybridGradientBoostingTree',
                            param_search_strategy='grid',
                            resampling_method='cv',
                            evaluation_metric='error_rate',
                            ref_metric=['auc'],
                            fold_num=5,
                            random_state=1,
                            param_values=cv_values)
hgc.fit(diabetes_train, key='ID', label='CLASS',
        partition_method='stratified',
        partition_random_state=1,
        stratified_column='CLASS')

In [ ]:
from hana_ml.model_storage import ModelStorage
#from hana_ml.model_storage_services import ModelSavingServices


# Creates an object called model_storage which must use the same connection with the model
model_storage = ModelStorage(connection_context=connection_context)
model_storage.clean_up()
# Saves the model for the first time
hgc.name = 'Model A'  # The model name is mandatory
hgc.version = 1
model_storage.save_model(model=hgc)

# Lists models
model_storage.list_models()

In [ ]:
# plan the schedule

model_storage.set_schedule(name='Model A',
                           version=1,
                           schedule_time='every 3 minutes',
                           connection_userkey='raymondyao',
                           init_params={"func" : 'HybridGradientBoostingTree',
                                        "param_search_strategy" : 'grid',
                                        "resampling_method" : 'cv',
                                        "evaluation_metric" : 'error_rate',
                                        "ref_metric" : ['auc'],
                                        "fold_num" : 5,
                                        "random_state" : 1,
                                        "param_values" : {'learning_rate': [0.1, 0.4, 0.7, 1],
                                                          'n_estimators': [4, 6, 8, 10],
                                                          'split_threshold': [0.1, 0.4, 0.7, 1]}},
                           fit_params={"key" : 'ID',
                                       "label" : 'CLASS',
                                       "partition_method" : 'stratified',
                                       "partition_random_state" : 1,
                                       "stratified_column" : 'CLASS'},
                           training_dataset_select_statement="SELECT * FROM DIABETES_TRAIN"
                          )

In [ ]:
model_storage.start_schedule('Model A', 1)

In [ ]:
model_storage.list_models().iat[0, 7]

In [ ]:
model_storage.terminate_schedule('Model A', 1)

In [ ]:
model_storage.list_models().iat[0, 7]

In [ ]:
from hana_ml.algorithms.pal.model_selection import GridSearchCV
from hana_ml.algorithms.pal.model_selection import RandomSearchCV
hgc2 = UnifiedClassification('HybridGradientBoostingTree')

gscv = GridSearchCV(estimator=hgc2, 
                    param_grid={'learning_rate': [0.1, 0.4, 0.7, 1],
                                'n_estimators': [4, 6, 8, 10],
                                'split_threshold': [0.1, 0.4, 0.7, 1]},
                    train_control=dict(fold_num=5,
                                       resampling_method='cv',
                                       random_state=1,
                                       ref_metric=['auc']),
                    scoring='error_rate')
gscv.fit(data=diabetes_train, key= 'ID',
         label='CLASS',
         partition_method='stratified',
         partition_random_state=1,
         stratified_column='CLASS',
         build_report=True)

# Evaluation

Let us compare cross-validation accuracy and test accuracy:

In [ ]:
cm = hgc.confusion_matrix_.collect()
cm

In [ ]:
gscv.estimator.confusion_matrix_.collect()

In [ ]:
train_accuracy = float(cm['COUNT'][cm['ACTUAL_CLASS']==cm['PREDICTED_CLASS']].sum())/cm['COUNT'].sum()
train_accuracy

In [ ]:
features = diabetes_train.columns
features.remove('CLASS')
features.remove('ID')
print(features)

In [ ]:
pred_res = hgc.predict(diabetes_test, key='ID', features=features)
pred_res.head(10).collect()
pred_res.dtypes()

In [ ]:
ts = diabetes_test.rename_columns({'ID': 'TID'}) .cast('CLASS', 'NVARCHAR(256)')
jsql = '{}."{}"={}."{}"'.format(pred_res.quoted_name, 'ID', ts.quoted_name, 'TID')
results_df = pred_res.join(ts, jsql, how='inner')
cm_df, classification_report_df = metrics.confusion_matrix(results_df, key='ID', label_true='CLASS', label_pred='SCORE') 

In [ ]:
import matplotlib.pyplot as plt
from hana_ml.visualizers.metrics import MetricsVisualizer
f, ax1 = plt.subplots(1,1)
mv1 = MetricsVisualizer(ax1)
ax1 = mv1.plot_confusion_matrix(cm_df, normalize=False)

In [ ]:
print("Recall, Precision and F_measures.")
classification_report_df.collect()

In [ ]:
_,_,_,metrics_res = hgc.score(data=diabetes_test, key='ID', label='CLASS')
metrics_res.collect()

In [ ]:
from hana_ml.model_storage import ModelStorage

model_storage = ModelStorage(connection_context=connection_context)
gscv.estimator.name = 'HGBT' 
gscv.estimator.version = 1
model_storage.save_model(model=gscv.estimator)
# Lists models
model_storage.list_models()

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport

mymodel = model_storage.load_model('HGBT', 1)
mymodel.fit(data=diabetes_train, key= 'ID',
            label='CLASS',
            partition_method='stratified',
            partition_random_state=1,
            stratified_column='CLASS',
            build_report=True)
UnifiedReport(mymodel).display()

In [ ]:
model_storage.delete_model('HGBT', 1)
connection_context.close()